# <center>Web of Science</center>

## Extracting data from the website using wos API

In [348]:
from wos import WosClient
import wos.utils
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import xmltodict, json
# SID = 1Wz1gsjtR2AKkXH1w5V

## Pull record of provided user

In [7]:
user='Ahmed, Amal'
with WosClient(user='HUSKY ID', password='Password',SID='4WuuQmMzkc2zaAQObdt') as client:
    data=(wos.utils.query(client, 'AU=%s'),user)
#     client.connect()
#     data=client.search('AU=%s',user)
    print(data)

('<?xml version="1.0" ?>\n<records>\n\t\n\n\t<REC r_id_disclaimer="ResearcherID data provided by Thomson Reuters">\n\t\t<UID>WOS:000076258100483</UID>\n\t\t<static_data>\n\t\t\t<summary>\n\t\t\t\t<EWUID>\n\t\t\t\t\t<WUID coll_id="WOS"/>\n\t\t\t\t\t<edition value="WOS.SCI"/>\n\t\t\t\t</EWUID>\n\t\t\t\t<pub_info coverdate="OCT 1998" has_abstract="N" issue="4" part_no="2" pubmonth="OCT" pubtype="Journal" pubyear="1998" sortdate="1998-10-01" supplement="S" vol="28">\n\t\t\t\t\t<page begin="283A" end="283A" page_count="1">283A-283A</page>\n\t\t\t\t</pub_info>\n\t\t\t\t<titles count="6">\n\t\t\t\t\t<title type="source">HEPATOLOGY</title>\n\t\t\t\t\t<title type="source_abbrev">HEPATOLOGY</title>\n\t\t\t\t\t<title type="abbrev_iso">Hepatology</title>\n\t\t\t\t\t<title type="abbrev_11">HEPATOLOGY</title>\n\t\t\t\t\t<title type="abbrev_29">HEPATOLOGY</title>\n\t\t\t\t\t<title type="item">African-American response rate to therapy for hepatitis C.</title>\n\t\t\t\t</titles>\n\t\t\t\t<names count="

## Sample of extracted XML

In [16]:
# infile = open(file)
# contents = infile.read()
contents=data[0]
soup = BeautifulSoup(contents,'xml')
titles = soup.find_all("abstract_text")

# soup.findChild("names")
for title in titles:
    print(title.get_text())
    


Reports the findings of an exploratory case study research on the contribution of internal and external communication to the commercial success of financial service innovations. An extensive case study research involving four innovation projects was conducted within a leading Belgian bank. The desk research and the 32 in-depth interviews with senior managers and project leaders served as a platform for theory development A propositional framework was developed that offers a contingency perspective regarding the role of communication during the different stages of the service innovation process. While many of the existing studies in the field have exclusively focused on external communication, the effectiveness of internal communication is a critical success factor Furthermore path dependency effects were created in the project life-cycle of the financial service innovation projects Finally, the findings suggest that the effectiveness of internal and external communication depends on t

## Converting XML to JSON for easy parsing

In [18]:
o = xmltodict.parse(contents)
Json=json.dumps(o) # '{"e": {"a": ["text", "text"]}}'
Json

'{"records": {"REC": [{"@r_id_disclaimer": "ResearcherID data provided by Thomson Reuters", "UID": "WOS:000076258100483", "static_data": {"summary": {"EWUID": {"WUID": {"@coll_id": "WOS"}, "edition": {"@value": "WOS.SCI"}}, "pub_info": {"@coverdate": "OCT 1998", "@has_abstract": "N", "@issue": "4", "@part_no": "2", "@pubmonth": "OCT", "@pubtype": "Journal", "@pubyear": "1998", "@sortdate": "1998-10-01", "@supplement": "S", "@vol": "28", "page": {"@begin": "283A", "@end": "283A", "@page_count": "1", "#text": "283A-283A"}}, "titles": {"@count": "6", "title": [{"@type": "source", "#text": "HEPATOLOGY"}, {"@type": "source_abbrev", "#text": "HEPATOLOGY"}, {"@type": "abbrev_iso", "#text": "Hepatology"}, {"@type": "abbrev_11", "#text": "HEPATOLOGY"}, {"@type": "abbrev_29", "#text": "HEPATOLOGY"}, {"@type": "item", "#text": "African-American response rate to therapy for hepatitis C."}]}, "names": {"@count": "7", "name": [{"@dais_id": "4713714", "@daisng_id": "10045011", "@role": "author", "@se

## Extracting required features

In [345]:
# extracting Abstract if present

def extractData(json_object):
    df=pd.DataFrame(columns=['S.No','Title','Authors','Published','Abstract','Publisher','Accession No'])
    for record in json_object['records']['REC']:
        temp_dict={}
        temp_dict['S.No']=1
        abstract=''
        k=record['static_data']['fullrecord_metadata']
        if 'abstracts' in k.keys():
            abstract=k['abstracts']['abstract']['abstract_text']['p']
        temp_dict['Abstract']=abstract


        titles=record['static_data']['summary']['titles']  # json type
    #     print (titles)
        temp=titles['title'][-1]#['#text']

        flag=0
        title=''
    #     print (temp)
        
        temp=titles['title'][-1]

        for key, value in temp.items():
            if (key=='@type' and value=='item'):
                flag=1
    #         print (key, value)            
            title=value

        if flag==0:
            title=''
        
        temp_dict['Title']=title

        author=record['static_data']['summary']['names']['name']
        temp=[]
        for i in range(0,len(author)):
            temp.append(author[i]['display_name'])
    #         print (author[i]['display_name'])
        author=temp
        temp_dict['Authors']=', '.join(author)
        pub_info=''
        pub_info=record['static_data']['summary']['pub_info']['@sortdate']
        temp_dict['Published']=pub_info
    #     print ("Publilcation info : ",pub_info)
        publisher_info=[]
        k=record['static_data']['summary']['publishers']['publisher']['names']['name']['full_name']
    #     print (k)
        l=record['static_data']['summary']['publishers']['publisher']['address_spec']['full_address']
        publisher_info.append(k+", "+l)
        temp_dict['Publisher']=' '.join(publisher_info)
        issn=''
    #     issn = record['dynamic_data']['cluster_related']['identifiers']['identifier'][0]['@value']
    #     temp_dict['ISSN']=issn
        accession_number=''
        accession_number= record['UID']
        temp_dict['Accession No']=accession_number
        df=df.append(temp_dict,ignore_index=True)
    return df.copy()

## Arranging the data into a DataFrame for user friendly display

In [349]:
df=extractData(json_object)
df.set_index('S.No', inplace=True)
df

,Title,Authors,Published,Abstract,Publisher,Accession No
S.No,,,,,,
1,African-American response rate to therapy for ...,"Fleckenstein, JF, Ismail, MK, Bockhold, K, Van...",1998-10-01,,"W B SAUNDERS CO, INDEPENDENCE SQUARE WEST CURT...",WOS:000076258100483
1,Linking communication to innovation success in...,"Lievens, A, Moenaert, RK, S'Jegers, R",1999-01-01,Reports the findings of an exploratory case st...,"MCB UNIV PRESS LTD, 60/62 TOLLER LANE, BRADFOR...",WOS:000079256600003
1,Association between Apolipoprotein E genotype ...,"McMichael, Gai L., Gibson, Catherine S., Goldw...",2008-11-01,Apolipoprotein E (APOE) plays a significant ro...,"SPRINGER, 233 SPRING ST, NEW YORK, NY 10013 USA",WOS:000260767100009
1,,"Yunfeng Wu, S. C. Ng, IEEE",2007-01-01,Early detection of suspicious breast lesions i...,"IEEE, 345 E 47TH ST, NEW YORK, NY 10017 USA",WOS:000252231900141
1,Light and Transmission Electronic Microscopy E...,"Farias, Roberta J. M., Sousa, Luciene B., S. L...",2008-08-01,[Purpose: Cornea storage for longer periods is...,"LIPPINCOTT WILLIAMS & WILKINS, 530 WALNUT ST, ...",WOS:000260055700009
